# Análise Estrutural & Elementos Finitos

In [1]:
import math
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.tri as mtri
import meshio
from scipy.spatial import Delaunay
import scipy as sp
import pandas as pd
import os                # Library used for system interaction (such as listing files in a directory,...)
import itertools as it   #biblioteca com funções de permutação a serem usadas na interação da matriz de conectividade com a global

## Funções de forma (ou interpolação)

In [2]:
#1D----------------------------------------------
##1ª ordem:
#Nh1 = [Nh11, Nh12, Nh13, Nh14]

def Nh11(x,l):
    return 2*(x/l)**3 - 3*(x/l)**2 + 1

def Nh12(x,l):
    return x**3/l**2 - 2*x**2/l + x

def Nh13(x,l):
    return -2*(x/l)**3 + 3*(x/l)**2

def Nh14(x,l):
    return x**3/l**2 - x**2/l

#Bh1 = [Bh11, Bh12, Bh13, Bh14]

def Bh11(x,l):
    return 6*(x/l)**2 - 6*(x/l)

def Bh12(x,l):
    return 3*(x/l)**2 - 4*x/l + 1

def Bh13(x,l):
    return -6*(x/l)**2 + 6*(x/l)

def Bh14(x,l):
    return 3*(x/l)**2 - 2*x/l

##  Viga sujeita a momento e cortante

$$ EI\frac{\partial^4 y}{\partial x^4} - p(x) = 0 $$


$$ \int_\Omega \left [ EI\frac{\partial^4 y}{\partial x^4} - p(x) \right ] \cdot \omega(x) \; d\Omega = 0 $$

Aplicando Toerema de Green para enfraquecer a equação:

$$ \int_{\Omega} EI\frac{\partial^3 y}{\partial x^3} \frac{\partial \omega(x)}{\partial x} \; d\Omega - \int_{\Gamma} EI\frac{\partial^3 y}{\partial x^3} \omega(x) \; d\Gamma - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

Assumindo C.C.'s de Dirichlet:

$$ \int_{\Omega} EI\frac{\partial^3 y}{\partial x^3} \frac{\partial \omega(x)}{\partial x} \; d\Omega - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

Repetindo o procedimento:

$$ \int_{\Omega} EI\frac{\partial^2 y}{\partial x^2} \frac{\partial^2 \omega(x)}{\partial x^2} \; d\Omega - \int_{\Gamma} EI\frac{\partial^2 y}{\partial x^2} \frac{\partial \omega(x)}{\partial x} \; d\Gamma - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

$$ \int_{\Omega} EI\frac{\partial^2 y}{\partial x^2} \frac{\partial^2 \omega(x)}{\partial x^2} \; d\Omega - \int_{\Omega} p(x) \omega(x) \; d\Omega = 0 $$

Discretizando a formulação fraca:

$$ \sum_{i}^{\infty} \sum_{j}^{\infty} \int_{\Omega} EI\frac{\partial^2 (N_i y_i)}{\partial x^2} \frac{\partial^2 (N_j \omega_j)}{\partial x^2} \; d\Omega - \sum_{i}^{\infty} \sum_{j}^{\infty} \int_{\Omega} N_k p_k \cdot N_j \omega_j \; d\Omega = 0 $$

$$ \sum_{i}^{\infty} \sum_{j}^{\infty} EI \int_{\Omega} B'_i B'_j \; d\Omega \cdot y_i \omega_j - \sum_{i}^{\infty} \sum_{j}^{\infty} \int_{\Omega} N_k N_j \; d\Omega \cdot p_k \omega_j = 0 $$

Dividindo a equação por $\omega_j$ e substituindo os somatórios duplos por somatórios de elemento, a equação assume a seguinte forma: 

$$ \sum_{e}^{\infty} EI \int_{\Omega} B'_i B'_j \; d\Omega \cdot y_i - \sum_{e}^{\infty} \int_{\Omega} N_k N_j \; d\Omega \cdot p_k = 0 $$

As integrais das funções de interpolação correspondem a matrizes elementares de rigidez e carga:

$$ K^e = EI \int_{\Omega} B'_i B'_j \; d\Omega \;\;,\;\;\;\; Q^e = \int_{\Omega} N_i N_j \; d\Omega $$

Fazendo a montagem das matrizes globais $K$ e $Q$:

$$ K \cdot y_i - Q \cdot p_k = 0 $$

In [ ]:
###Definindo nº de elementos e nós 

nElementos = len(malha)-1
nNos = nElementos+1

nNosLocais = 2

K11 = lambda x,X1,X2 : B11(x,X1,X2)*k*B11(x,X1,X2)
K12 = lambda x,X1,X2 : B11(x,X1,X2)*k*B12(x,X1,X2)
K21 = lambda x,X1,X2 : B12(x,X1,X2)*k*B11(x,X1,X2)
K22 = lambda x,X1,X2 : B12(x,X1,X2)*k*B12(x,X1,X2)

G1 = lambda x,X1,X2 : S(x)*N11(x,X1,X2)
G2 = lambda x,X1,X2 : S(x)*N12(x,X1,X2)

M11 = lambda x,X1,X2 : N11(x,X1,X2)*N11(x,X1,X2)
M12 = lambda x,X1,X2 : N11(x,X1,X2)*N12(x,X1,X2)
M21 = lambda x,X1,X2 : N12(x,X1,X2)*N11(x,X1,X2)
M22 = lambda x,X1,X2 : N12(x,X1,X2)*N12(x,X1,X2)

###Função de integração a ser usada
def INTEGRACAO_1(f,X1,X2):
    return sp.integrate.quad(f,X1,X2,(X1,X2))[0]

##Integrando e montando
listaKe = np.empty((2,2,nElementos))
listaGe = np.empty((1,2,nElementos))
listaMe = np.empty((2,2,nElementos))

for e in range(0,nElementos):
    Ke = np.array([[INTEGRACAO_1(K11,malha[e],malha[e+1]),INTEGRACAO_1(K12,malha[e],malha[e+1])],
                [INTEGRACAO_1(K21,malha[e],malha[e+1]),INTEGRACAO_1(K22,malha[e],malha[e+1])]])    

    Ge = np.array([[INTEGRACAO_1(G1,malha[e],malha[e+1]),INTEGRACAO_1(G2,malha[e],malha[e+1])]])

    Me = np.array([[INTEGRACAO_1(M11,malha[e],malha[e+1]),INTEGRACAO_1(M12,malha[e],malha[e+1])],
                [INTEGRACAO_1(M21,malha[e],malha[e+1]),INTEGRACAO_1(M22,malha[e],malha[e+1])]]) 

    listaKe[:,:,e] = Ke
    listaGe[:,:,e] = Ge
    listaMe[:,:,e] = Me

In [ ]:
# parametros da simulacao

## malha
L = 1600.0   # m
nx = 6400
ne = nx-1

## CCs
Yo = 1.0
Yf = 0.0
Thetao = 0.0
Thetaf = 1.0

# propriedades da linha
d = 180.0  # mm
t = 22.0   # mm
D = d + 2*t   # mm
I = np.pi*(D**4 - d**4)/64
E = 206*10**9

# transiente
to = 0.0
tf = 2.0
dt = 0.01
nt = int((tf-to)/dt+1)

# carga
def S(x):
    return (50.0/3.0)*(x+2.0)

#beta = 0.0 # metodo explicito
beta = 1.0 # metodo implicito
#beta = 0.5 # metodo Crank-Nicolson

# geracao de malha 1D
X = np.linspace(0,L,nx)

IEN = np.zeros( (ne,2),dtype='int' )
for e in range(0,ne):
 IEN[e] = [e,e+1]

# lista de indices de condicao de contorno
cc = [0,nx-1]

# vetor com valores das condicoes de contorno
Ycc = np.zeros( (nx),dtype='float' )
Ycc[0] = Yo
Ycc[-1] = Yf

Thetacc = np.zeros( (nx),dtype='float' )
Thetacc[0] = Thetao
Thetacc[-1] = Thetaf

# inicializacao das matrizes globais
K = np.zeros( (nx,nx),dtype='float' )
Q = np.zeros( (nx,nx),dtype='float' )

# obtendo matrizes elementares
## funcoes de interpolacao

K11 = lambda x,l : EI*Bh11(x,l)*Bh11(x,l)
K12 = lambda x,l : EI*Bh11(x,l)*Bh12(x,l)
K13 = lambda x,l: EI*Bh11(x,l)*Bh13(x,l)
K14 = lambda x,l : EI*Bh11(x,l)*Bh14(x,l)
K21 = lambda x,l : EI*Bh12(x,l)*Bh11(x,l)
K22 = lambda x,l : EI*Bh12(x,l)*Bh12(x,l)
K23 = lambda x,l : EI*Bh12(x,l)*Bh13(x,l)
K24 = lambda x,l : EI*Bh12(x,l)*Bh14(x,l)
K31 = lambda x,l : EI*Bh13(x,l)*Bh11(x,l)
K32 = lambda x,l : EI*Bh13(x,l)*Bh12(x,l)
K33 = lambda x,l : EI*Bh13(x,l)*Bh13(x,l)
K34 = lambda x,l : EI*Bh13(x,l)*Bh14(x,l)
K41 = lambda x,l : EI*Bh14(x,l)*Bh11(x,l)
K42 = lambda x,l : EI*Bh14(x,l)*Bh12(x,l)
K43 = lambda x,l : EI*Bh14(x,l)*Bh13(x,l)
K44 = lambda x,l : EI*Bh14(x,l)*Bh14(x,l)

Q11 = lambda x,l : Nh11(x,l)*Nh11(x,l)
Q12 = lambda x,l : Nh11(x,l)*Nh12(x,l)
Q13 = lambda x,l : Nh11(x,l)*Nh13(x,l)
Q14 = lambda x,l : Nh11(x,l)*Nh14(x,l)
Q21 = lambda x,l : Nh12(x,l)*Nh11(x,l)
Q22 = lambda x,l : Nh12(x,l)*Nh12(x,l)
Q23 = lambda x,l : Nh12(x,l)*Nh13(x,l)
Q24 = lambda x,l : Nh12(x,l)*Nh14(x,l)
Q31 = lambda x,l : Nh13(x,l)*Nh11(x,l)
Q32 = lambda x,l : Nh13(x,l)*Nh12(x,l)
Q33 = lambda x,l : Nh13(x,l)*Nh13(x,l)
Q34 = lambda x,l : Nh13(x,l)*Nh14(x,l)
Q41 = lambda x,l : Nh14(x,l)*Nh11(x,l)
Q42 = lambda x,l : Nh14(x,l)*Nh12(x,l)
Q43 = lambda x,l : Nh14(x,l)*Nh13(x,l)
Q44 = lambda x,l : Nh14(x,l)*Nh14(x,l)

## Funcao de integracao a ser usada
def INTEGRACAO_1(f,e):
     v1,v2 = IEN[e]
     l = X[v2]-X[v1]
     
     return sp.integrate.quad(f,X[v1],X[v2],l)[0]

## Integrando
kelem = np.empty((2,2,ne))
qelem = np.empty((1,2,ne))

for e in range(0,ne):
    Qe = np.array([[INTEGRACAO_1(K11,e),INTEGRACAO_1(K12,e),INTEGRACAO_1(K13,e),INTEGRACAO_1(K14,e)],
                   [INTEGRACAO_1(K21,e),INTEGRACAO_1(K22,e),INTEGRACAO_1(K23,e),INTEGRACAO_1(K24,e)],
                   [INTEGRACAO_1(K31,e),INTEGRACAO_1(K32,e),INTEGRACAO_1(K33,e),INTEGRACAO_1(K34,e)]
                   [INTEGRACAO_1(K41,e),INTEGRACAO_1(K42,e),INTEGRACAO_1(K43,e),INTEGRACAO_1(K44,e)]])     

    Qe = np.array([[INTEGRACAO_1(Q11,e),INTEGRACAO_1(Q12,e),INTEGRACAO_1(Q13,e),INTEGRACAO_1(Q14,e)],
                   [INTEGRACAO_1(Q21,e),INTEGRACAO_1(Q22,e),INTEGRACAO_1(Q23,e),INTEGRACAO_1(Q24,e)],
                   [INTEGRACAO_1(Q31,e),INTEGRACAO_1(Q32,e),INTEGRACAO_1(Q33,e),INTEGRACAO_1(Q34,e)]
                   [INTEGRACAO_1(Q41,e),INTEGRACAO_1(Q42,e),INTEGRACAO_1(Q43,e),INTEGRACAO_1(Q44,e)]]) 

    kelem[:,:,e] = Ke
    qelem[:,:,e] = Qe

for e in range(0,ne):

 for ilocal in range(0,2):
  iglobal = IEN[e,ilocal]
  for jlocal in range(0,2):
   jglobal = IEN[e,jlocal]

   K[iglobal,jglobal] += kelem[ilocal,jlocal,e]
   Q[iglobal,jglobal] += qelem[ilocal,jlocal,e]

# definicao da condicao inicial
Y = np.zeros( (nx),dtype='float' )
Theta = np.zeros( (nx),dtype='float' )

for i in cc:
 Y[i] = Ycc[i]
 Theta[i] = Thetacc[i]

# definicao do sistema linear
qvec = (alpha*Q/k)*np.ones( (nx),dtype='float' )

##A = M/dt + K        # metodo implicito para difusao (beta = 1)
##A = M/dt            # metodo explicito para difusao (beta = 0)
##A = M/dt + beta*K   # metodo hibrido para difusao   (beta = 0.5)

A = M/dt + beta*K

# imposicao das c.c.s de Dirichlet na matriz A
for i in cc:
 A[i,:] = 0.0 # zerar linha
 A[i,i] = 1.0 # incluir 1 na diagonal

# recorrência temporal
for n in range(0,nt):
 ##b = (M/dt)@T + M@qvec                  # metodo implicito para difusao (beta = 1)
 ##b = (M/dt - K)@T + M@qvec              # metodo explicito para difusao (beta = 0)
 ##b = (M/dt - (1.0-beta)*K)@T + M@qvec   # metodo hibrido para difusao   (beta = 0.5)

 b = (M/dt - (1.0-beta)*K)@T + M@qvec

 # imposicao das c.c.s de Dirichlet no vetor b
 for i in cc:
  b[i]   = Tcc[i]

 # temperatura em n+1
 T = np.linalg.solve(A,b)

 plt.scatter(X,T,s=1.5,color='b')
 plt.plot(X,T,'b-',linewidth=1.0)
plt.show()